In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
from catboost import CatBoostClassifier
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/playground-series-s5e7/train.csv')


In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.nunique()

In [ ]:
col_list = df.columns.to_list()
col_list

In [ ]:
col_to_remove = ['id']
list_for_value_count = [x for x in col_list if x not in col_to_remove]
for i in list_for_value_count:
    print(df[i].value_counts())

In [ ]:
df.duplicated().sum()

In [ ]:
df.isnull().sum().sort_values(ascending=False)

In [ ]:
df['Personality'].value_counts().plot(kind='pie', legend=True)
round(df['Personality'].value_counts(normalize=True)*100,2)

In [ ]:

df.groupby(['Time_spent_Alone','Personality']).size().unstack().plot(kind='bar', stacked=True)
df.groupby('Time_spent_Alone')['Personality'].value_counts(normalize=True).unstack().round(2)*100


In [ ]:
for i in list_for_value_count:
    df.groupby([i,'Personality']).size().unstack().plot(kind='bar', stacked=True, title=i)

In [ ]:
df[['Time_spent_Alone','Social_event_attendance','Going_outside','Friends_circle_size','Post_frequency']].corr()

In [ ]:
for column in df.columns:
    mode_value = df[column].mode(dropna=True)
    if not mode_value.empty:
        df[column].fillna(mode_value[0], inplace=True)

In [ ]:
df.isnull().sum().sort_values(ascending=False)

In [ ]:
col_list.remove('id')
col_list.remove('Personality')

In [ ]:
col_list

In [ ]:
test_data = pd.read_csv('/kaggle/input/playground-series-s5e7/test.csv')
test_X = test_data[col_list].copy()
for column in test_X.columns:
    mode_value = test_X[column].mode(dropna=True)
    if not mode_value.empty:
        test_X[column].fillna(mode_value[0], inplace=True)
print(test_X.isnull().sum().sort_values(ascending=False))
test_X.head(5)

In [ ]:
df_train = df.drop(['id','Personality'], axis=1).copy()
df_test = test_X.copy()
y = df['Personality'].copy()
categorical_cols = col_list

In [ ]:
# Zakładam, że masz: 
# df_train — dane treningowe (bez SalePrice)
# df_test — dane testowe (do predykcji)
# y — target
# numerical_cols — lista kolumn numerycznych
# categorical_cols — lista kolumn kategorycznych

# 1. Wyciągamy tylko kolumny

X_train_cat = pd.get_dummies(df_train[categorical_cols], drop_first=True)
X_test_cat  = pd.get_dummies(df_test[categorical_cols], drop_first=True)

# 2. Dopasuj kolumny OneHotEncodera między train i test
X_train_cat, X_test_cat = X_train_cat.align(X_test_cat, join="left", axis=1, fill_value=0)

# 3. Sklejamy numeryczne + zakodowane kategorie
X_train_final = X_train_cat
X_test_final  = X_test_cat



In [ ]:
X_test_final.head(1)

In [ ]:
col_float= ['Time_spent_Alone','Social_event_attendance','Going_outside',
              'Friends_circle_size','Post_frequency']

In [ ]:
X_train_final[col_float]=X_train_final[col_float].astype('Int64')
X_test_final[col_float]=X_test_final[col_float].astype('Int64')
X_test_final.head(1)

In [ ]:
cat_features = X_test_final.columns.to_list()

In [ ]:
# 4. Trenujemy model

model = CatBoostClassifier(
                            cat_features=cat_features,
                            iterations=1000, #1000 najlepszy
                            learning_rate=0.02, #0.05 najlepszy
                            depth=6,
                            auto_class_weights = None,   # najlepszy'Balanced',
                            verbose=0
                          )

model.fit(X_train_final, y)

# 5. Predykcja na test.csv (do submission)
preds = model.predict(X_test_final)

In [ ]:
df_test = pd.read_csv('/kaggle/input/playground-series-s5e7/test.csv')

submission = pd.DataFrame({
    "id": df_test["id"],
    "Personality": preds
})
submission.to_csv("submission.csv", index=False)

print('Sukcesss!!!')